In [4]:
import json
import glob
import numpy as np
import os
import cv2
def PolyArea(x,y):
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))
def create_folder(dirr):
    try:
        os.mkdir(dirr)
    except:
        pass

def extract_data(json_file):
    with open(json_file) as f:
        file=json.load(f)
        width=file['imageWidth']
        height=file['imageHeight']
        path=file['imagePath'].split('\\')[-1]
        shapes=file['shapes']
        img_dict={"filename":path,"size":width*height,"regions":[]}
        for shape in shapes:
            pts=[]
            xs=[]
            ys=[]
            bbox=[]
            ID=shape['label']
            for j in shape['points']:
                xs.append(float(j[0]))
                ys.append(float(j[1]))
                pts.append(float(j[0]))
                pts.append(float(j[1]))
                
            img_dict["regions"].append({"shape_attributes": {
          "name": "polygon",
          "all_points_x": xs,
          "all_points_y": ys},
        "region_attributes": {"stock":ID}
                                        })
                
    return img_dict,path
           

In [5]:
def convert_files(dirr,val_split):
    jsons=glob.glob(f'{dirr}/*.json')
    create_folder('images')
    create_folder('images/train')
    create_folder('images/val')
    train_json=dict()
    val_json=dict()
    n_train=int(len(jsons)*(1-val_split))
    print(f"Training images found {n_train}")
    print(f"Validation images found {len(jsons)-n_train}")

    for ind,i in enumerate(jsons):            
        if ind<=(n_train):
            img_dict,img_dir = extract_data(i)
            img=cv2.imread(f'{dirr}/{img_dir}')
            cv2.imwrite(f'images/train/{img_dir}',img)

            train_json[img_dir]=img_dict

        else:
            img_dict,img_dir = extract_data(i)
            img=cv2.imread(f'{dirr}/{img_dir}')
            cv2.imwrite(f'images/val/{img_dir}',img)

            val_json[img_dir]=img_dict

    with open('images/train/train.json','w') as fp:
        json.dump(train_json,fp)

    with open('images/val/val.json','w') as fp:
        json.dump(val_json,fp)


In [6]:
convert_files('test',0.2)

Training images found 31
Validation images found 8
